In [1]:
## Definición del problema
##   Se desea contar la frecuencia de ocurrencia de palabras en un conjunto de documentos. Debido a los requerimientos de diseño (gran volúmen de datos y tiempos rápidos de respuesta) 
##   se desea implementar una arquitectura Big Data. Se desea implementar la solución en Python y ejecutar Hadoop en modo streamming.

In [2]:
## Archivos de prueba
##   A continuación se generarán tres archivos de prueba para probar el sistema. Puede usar directamente comandos del sistema operativo en el Terminal 
##   y el editor de texto pico para crear los archivos.

In [3]:
## Se crea el directorio de entrada
!rm -rf input output
!mkdir input

In [4]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns
in data. Especially valuable in areas rich with recorded information, analytics relies
on the simultaneous application of statistics, computer programming and operations research
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business
performance. Specifically, areas within analytics include predictive analytics, prescriptive
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,
marketing optimization and marketing mix modeling, web analytics, call analytics, speech
analytics, sales force sizing and optimization, price and promotion modeling, predictive
science, credit risk analysis, and fraud analytics. Since analytics can require extensive
computation (see big data), the algorithms and software used for analytics harness the most
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [5]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to
research potential trends, to analyze the effects of certain decisions or events, or to
evaluate the performance of a given tool or scenario. The goal of analytics is to improve
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [6]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions
about the information they contain, increasingly with the aid of specialized systems
and software. Data analytics technologies and techniques are widely used in commercial
industries to enable organizations to make more-informed business decisions and by
scientists and researchers to verify or disprove scientific models, theories and
hypotheses.

Writing input/text2.txt


In [7]:
## Prueba de la implementación fuera de Hadoop
##   Para realizar la implementación solicitada se usará Hadoop Streaming, el cual permite la implementación de aplicaciones MapReduce en cualquier lenguaje que soporte 
##   la creación de scripts. Para este caso se ejemplificará la implementación en Python. Hadoop streaming debe su nombre a que usa los streamings de Unix para la entrada 
##   y salida en texto (stdin, stdout y stderr respectivamente). El sistema lee y escribe líneas de texto una a una a los respectivos flujos de entrada y salida, tal como 
##   ocurre de forma estándar en las herramientas de la línea de comandos de Linux.
##
##   La implementación requiere un programa para realizar el mapeo y otro para la reducción que se ejecutan independientemente; Hadoop se encarga de la coordinación entre ellos. 
##   El intercambio de información se hace a traves de texto, que es el lenguaje universal para intercambio de información.

In [8]:
## Paso 1
##   Se implementa la función map en Python y se guarda en el archivo mapper.py.

In [9]:
%%writefile mapper.py
#! /usr/bin/python3

##
## Esta es la funcion que mapea la entrada a parejas (clave, valor)
##
import sys
if __name__ == "__main__":

    ##
    ## itera sobre cada linea de codigo recibida
    ## a traves del flujo de entrada
    ##
    for line in sys.stdin:

        ##
        ## genera las tuplas palabra \tabulador 1
        ## ya que es un conteo de palabras
        ##
        for word in line.split():

            ##
            ## escribe al flujo estandar de salida
            ##
            sys.stdout.write("{}\t1\n".format(word))

Writing mapper.py


In [10]:
## Paso 2

In [11]:
## El programa anterior se hace ejecutable
!chmod +x mapper.py

In [12]:
## la salida de la función anterior es:
!cat ./input/text*.txt | ./mapper.py | head

Analytics	1
is	1
the	1
discovery,	1
interpretation,	1
and	1
communication	1
of	1
meaningful	1
patterns	1


In [13]:
## Paso 3
##   Se implementa la función reduce y se salva en el archivo reducer.py.

In [22]:
%%writefile reducer.py
#! /usr/bin/python3

import sys

##
## Esta funcion reduce los elementos que
## tienen la misma clave
##

if __name__ == '__main__':

    curkey = None
    total = 0

    ##
    ## cada linea de texto recibida es una
    ## entrada clave \tabulador valor
    ##
    for line in sys.stdin:

        key, val = line.split("\t")
        val = int(val)

        if key == curkey:
            ##
            ## No se ha cambiado de clave. Aca se
            ## acumulan los valores para la misma clave.
            ##
            total += val
        else:
            ##
            ## Se cambio de clave. Se reinicia el
            ## acumulador.
            ##
            if curkey is not None:
                ##
                ## una vez se han reducido todos los elementos
                ## con la misma clave se imprime el resultado en
                ## el flujo de salida
                ##
                sys.stdout.write("{}\t{}\n".format(curkey, total))

            curkey = key
            total = val

    sys.stdout.write("{}\t{}\n".format(curkey, total))

Overwriting reducer.py


In [24]:
## Paso 4

In [25]:
## El archivo se hace ejecutable
!chmod +x reducer.py

In [26]:
##  Paso 5
##    Se realiza la prueba de la implementación en el directorio actual antes de realizar la ejecución en el modo pseudo-distribuido

In [27]:
##
## La función sort hace que todos los elementos con
## la misma clave queden en lineas consecutivas.
## Hace el papel del módulo Shuffle & Sort
##
!cat ./input/text*.txt | ./mapper.py | sort | ./reducer.py | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


In [28]:
###############################################################################################################################
## Ejecución en Hadoop en modo pseudo-distribuido
##   Una vez se tienen las versiones anteriores funcionando, se puede proceder a ejecutar la tarea directamente en Hadoop.

In [29]:
## Paso 1. Movimiento de los datos al HDFS
##   Se copian los archivos de entrada del sistema local (o del datalake) al HDFS.

In [30]:
!hadoop fs -mkdir input
!hadoop fs -copyFromLocal input/* input
!hadoop fs -ls input/*

-rw-r--r--   1 root supergroup       1082 2019-11-27 23:33 input/text0.txt
-rw-r--r--   1 root supergroup        349 2019-11-27 23:33 input/text1.txt
-rw-r--r--   1 root supergroup        435 2019-11-27 23:33 input/text2.txt


In [31]:
## Vaya a la pantalla de monitoreo del NameNode (http://127.0.0.1:50070/) y ubique los archivos en el HDFS.
## La gestión de archivos entre el sistema local y el HDFS se realiza mediante comandos similares a los del sistema operativo Unix en Terminal. A continuación se resumen los principales comandos.
## hadoop fs -help: Imprime la ayuda en pantalla para todos los comandos.
## Comandos para la gestion de directorios y archivos.
##  hadoop fs -ls <path>
##  hadoop fs -mkdir <path>
##  hadoop fs -rmdir <path>
##  hadoop fs -cp <src> <dest>
##  hadoop fs -mv <src> <dest>
##  hadoop fs -rm <path>
##  hadoop fs -cat <path>
##  hadoop fs -head <path>
##  hadoop fs -tail <path>
##  hadoop fs -text <path>. Imprime el arachivo en <path> y lo imprime en formato texto. Soporta archivos zip, TextRecordInputStream y Avro.
##  hadoop fs -stat <path>: Imprime estadísticos de <path>.
## 
## Transferencia de información entre el sistema local y el HDFS.
##  hadoop fs -get <src> <localdest> / hadoop fs -copyToLocal <src> <localdest>. Copia el contenido de <src> en el HDFS en <localdest> en el sistema local.
##  hadoop fs -put <localsrc> <dest> / hadoop fs -copyFromLocal <src> <localdest>. Copia el contenido de <localsrc> en el sistema local a <dest> en el HDFS.
##  hadoop fs -count <path>. Cuenta el número de directorios, archivos y bytes en <path>.
##  hadoop fs -appendToFile <localsrc> <dest>: pega al final de <dest> el contenido de los archivos en <localsrc>.

In [32]:
## Paso 2
##   Se ejecuta Hadoop Streaming en modo pseudo-distribuido.

In [33]:
## Se ejecuta en Hadoop.
##   -files: archivos a copiar al hdfs
##   -input: archivo de entrada
##   -output: directorio de salida
##   -file: archivos a copiar de la maquina local al hdfs
##   -maper: programa que ejecuta el map
##   -reducer: programa que ejecuta la reducción
##
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar -files mapper.py,reducer.py  -input input  -output output -mapper mapper.py -reducer reducer.py

packageJobJar: [/tmp/hadoop-unjar9048060615756020707/] [] /tmp/streamjob7853660026548404963.jar tmpDir=null


In [34]:
## Alternativamente, podria usar:
## hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
##   -files mapper.py,reducer.py  \
##   -input input \
##   -output output \
##   -mapper 'python3 $PWD/mapper.py' \
##   -reducer 'python3 $PWD/reducer.py'

In [35]:
## contenido del directorio con los
## resultados de la corrida
!hadoop fs -ls output

Found 2 items
-rw-r--r--   1 root supergroup          0 2019-11-27 23:35 output/_SUCCESS
-rw-r--r--   1 root supergroup       1649 2019-11-27 23:35 output/part-00000


In [36]:
## se visualiza el archivo con los
## resultados de la corrida
!hadoop fs -cat output/part-00000 | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


In [37]:
## Paso 3 (Movimiento de los resultados al sistema local)

In [38]:
!hadoop fs -copyToLocal output output
!ls output/*

output/_SUCCESS  output/part-00000


In [39]:
## Paso 4 (Limpieza del hdfs)

In [40]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rmdir input output

Deleted input/text0.txt
Deleted input/text1.txt
Deleted input/text2.txt
Deleted output/_SUCCESS
Deleted output/part-00000


In [41]:
## Limpieza de la máquina local

In [42]:
!rm reducer.py mapper.py
!rm -rf input output